# O/D Efficiency

Efficiency is calculated by +1 for being on the line when we score and -1 for being on the line when we get scored on, then normalized.

Use 'Points vs. ...' csv for what happened on each point and 'Player Stats vs. ...' for who was on the line for that point

In [2]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [3]:
player_stats_df = pd.read_csv('WY-Wolves/Player Stats vs. YK-Buddha 2017-03-11_14-30.csv')
points_df = pd.read_csv('WY-Wolves/Points vs. YK-Buddha 2017-03-11_14-30.csv')

In [4]:
# get the points each player played in

players = player_stats_df.loc[:, ['Player', 'Points played']]
print(players)

              Player                           Points played
0         Joyce Wang  1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1   #22 Paul Simpson  1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
2   #27 Daniel Zhang  1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
3         Cecilia Wu  1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
4    #69 Justin Chen           1,2,3,7,8,9,10,12,13,14,15,16
5         Jamie Chen                  1,2,3,4,5,6,9,10,13,14
6           Tina Zhu                 3,4,5,6,7,8,11,12,15,16
7       #19 Ben Wang                              2,11,15,16
8      Theresa Zhang                              9,10,13,14
9     #66 Jerry Chen                                    5,12
10         Alex Chen                                    6,11
11         Oscar Yao                                       8
12          Jack Xia                                       4
13         Julie Luo                                       1
14      #41 Billy Ni                                       7


In [5]:
# find the o_points, the ones we scored on and the ones we got scored on

o_points = points_df[points_df['Started on offense?'] == 1]
o_points_won = o_points[o_points['Scored?'] == 1]
o_points_lost = o_points[o_points['Scored?'] == 0]

In [6]:
# dictionary with each o point that we scored
# keys are the point numbers
# values are lists of players on the line that point

points_won = o_points_won.loc[:, ['Point']].values

o_lines_plus = {}

if points_won.size > 0:
    for point in np.nditer(points_won):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if point in o_lines_plus.keys():
                    o_lines_plus[point].append(row[1][0])
                else:
                    o_lines_plus[point] = [row[1][0]]

for point, line in o_lines_plus.items():
    print('Point',point)
    for player in line:
        print('\t', player)
else:
    print({})

Point 3
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Jamie Chen
	 Tina Zhu
Point 7
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Tina Zhu
	 #41 Billy Ni
Point 9
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Jamie Chen
	 Theresa Zhang
Point 13
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Jamie Chen
	 Theresa Zhang
Point 15
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Tina Zhu
	 #19 Ben Wang
{}


In [7]:
# dictionary with each o point that we got scored on
# keys are the point numbers
# values are lists of players on the line that point

points_lost = o_points_lost.loc[:,['Point']].values

o_lines_minus = {}

if points_lost.size > 0:
    for point in np.nditer(points_lost):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if point in o_lines_minus.keys():
                    o_lines_minus[point].append(row[1][0])
                else:
                    o_lines_minus[point] = [row[1][0]]
                    

for point, line in o_lines_minus.items():
    print('Point',point)
    for player in line:
        print('\t', player)
else:
    print({})

Point 2
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Jamie Chen
	 #19 Ben Wang
{}


In [8]:
# get the number of o points played for each player as a dictionary
# keys are player names
# values are number of o points played

points_played = o_points.loc[:,['Point']].values
o_points_played = {}
if points_played.size > 0:
    for point in np.nditer(points_played):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if row[1][0] in o_points_played.keys():
                    o_points_played[row[1][0]] += 1
                else:
                    o_points_played[row[1][0]] = 1
                    
for player, points in o_points_played.items():
    print(player, ':\t', points)

Joyce Wang :	 6
#22 Paul Simpson :	 6
#27 Daniel Zhang :	 6
Cecilia Wu :	 6
#69 Justin Chen :	 6
Jamie Chen :	 4
#19 Ben Wang :	 2
Tina Zhu :	 3
#41 Billy Ni :	 1
Theresa Zhang :	 2


In [10]:
# calculate offensive efficiency as a dictionary
# keys are player names
# values are their effiencies between -1 and 1

# create the dictionary and add all the positives
o_eff = {}
for point, line in o_lines_plus.items():
    for player in line:
        if player in o_eff.keys():
            o_eff[player] += 1
        else:
            o_eff[player] = 1

# add all the minuses
for point, line in o_lines_minus.items():
    for player in line:
        if player in o_eff.keys():
            o_eff[player] -= 1
        else:
            o_eff[player] = -1

# divide by the number of o points played
for player, eff in o_eff.items():
    o_eff[player] = eff/o_points_played[player]
    
for player, eff in o_eff.items():
    print(player, ':\t', eff)

Joyce Wang :	 0.6666666666666666
#22 Paul Simpson :	 0.6666666666666666
#27 Daniel Zhang :	 0.6666666666666666
Cecilia Wu :	 0.6666666666666666
#69 Justin Chen :	 0.6666666666666666
Jamie Chen :	 0.5
Tina Zhu :	 1.0
#41 Billy Ni :	 1.0
Theresa Zhang :	 1.0
#19 Ben Wang :	 0.0


In [11]:
# find the d_points, the ones we scored on and the ones we got scored on

d_points = points_df[points_df['Started on offense?'] == 0]
d_points_won = d_points[d_points['Scored?'] == 1]
d_points_lost = d_points[d_points['Scored?'] == 0]

In [12]:
# dictionary with each d point that we scored
# keys are the point numbers
# values are lists of players on the line that point

points_won = d_points_won.loc[:, ['Point']].values

d_lines_plus = {}

if points_won.size > 0:
    for point in np.nditer(points_won):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if point in d_lines_plus.keys():
                    d_lines_plus[point].append(row[1][0])
                else:
                    d_lines_plus[point] = [row[1][0]]

for point, line in d_lines_plus.items():
    print('Point',point)
    for player in line:
        print('\t', player)
else:
    print({})

Point 4
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 Jamie Chen
	 Tina Zhu
	 Jack Xia
Point 5
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 Jamie Chen
	 Tina Zhu
	 #66 Jerry Chen
Point 10
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Jamie Chen
	 Theresa Zhang
Point 11
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 Tina Zhu
	 #19 Ben Wang
	 Alex Chen
Point 16
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Tina Zhu
	 #19 Ben Wang
{}


In [13]:
# dictionary with each d point that we got scored on
# keys are the point numbers
# values are lists of players on the line that point

points_lost = d_points_lost.loc[:,['Point']].values

d_lines_minus = {}

if points_lost.size > 0:
    for point in np.nditer(points_lost):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if point in d_lines_minus.keys():
                    d_lines_minus[point].append(row[1][0])
                else:
                    d_lines_minus[point] = [row[1][0]]
                    

for point, line in d_lines_minus.items():
    print('Point',point)
    for player in line:
        print('\t', player)
else:
    print({})

Point 1
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Jamie Chen
	 Julie Luo
Point 6
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 Jamie Chen
	 Tina Zhu
	 Alex Chen
Point 8
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Tina Zhu
	 Oscar Yao
Point 12
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Tina Zhu
	 #66 Jerry Chen
Point 14
	 Joyce Wang
	 #22 Paul Simpson
	 #27 Daniel Zhang
	 Cecilia Wu
	 #69 Justin Chen
	 Jamie Chen
	 Theresa Zhang
{}


In [14]:
# get the number of d points played for each player as a dictionary
# keys are player names
# values are number of o points played

points_played = d_points.loc[:,['Point']].values
d_points_played = {}
if points_played.size > 0:
    for point in np.nditer(points_played):
        point = int(point)
        for row in players.iterrows():
            player_points = row[1][1].split(',')
            if str(point) in player_points:
                if row[1][0] in d_points_played.keys():
                    d_points_played[row[1][0]] += 1
                else:
                    d_points_played[row[1][0]] = 1
                    
for player, points in d_points_played.items():
    print(player, ':\t', points)

Joyce Wang :	 10
#22 Paul Simpson :	 10
#27 Daniel Zhang :	 10
Cecilia Wu :	 10
#69 Justin Chen :	 6
Jamie Chen :	 6
Julie Luo :	 1
Tina Zhu :	 7
Jack Xia :	 1
#66 Jerry Chen :	 2
Alex Chen :	 2
Oscar Yao :	 1
Theresa Zhang :	 2
#19 Ben Wang :	 2


In [15]:
# calculate defensive efficiency as a dictionary
# keys are player names
# values are their effiencies between -1 and 1

# create the dictionary and add all the positives

d_eff = {}
for point, line in d_lines_plus.items():
    for player in line:
        if player in d_eff.keys():
            d_eff[player] += 1
        else:
            d_eff[player] = 1

# subtract all the negatives
for point, line in d_lines_minus.items():
    for player in line:
        if player in d_eff.keys():
            d_eff[player] -= 1
        else:
            d_eff[player] = -1

# divide by the number of points played
for player, eff in d_eff.items():
    d_eff[player] = eff/d_points_played[player]
    
for player, eff in d_eff.items():
    print(player, ':\t', eff)

Joyce Wang :	 0.0
#22 Paul Simpson :	 0.0
#27 Daniel Zhang :	 0.0
Cecilia Wu :	 0.0
Jamie Chen :	 0.0
Tina Zhu :	 0.14285714285714285
Jack Xia :	 1.0
#66 Jerry Chen :	 0.0
#69 Justin Chen :	 -0.3333333333333333
Theresa Zhang :	 0.0
#19 Ben Wang :	 1.0
Alex Chen :	 0.0
Julie Luo :	 -1.0
Oscar Yao :	 -1.0
